In [ ]:
#default_exp optimizers

In [ ]:
from nbdev.export import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
import math

import torch
from fvcore.common import registry
from omegaconf import DictConfig
from torch.optim.optimizer import Optimizer, required

from src import _logger

## Create Optimizer Registery -

In [ ]:
# export
OPTIM_REGISTERY = registry.Registry("Optimizers")

In [ ]:
# export
# register all default optimizers
OPTIM_REGISTERY.register(torch.optim.SGD)
OPTIM_REGISTERY.register(torch.optim.RMSprop)
OPTIM_REGISTERY.register(torch.optim.Adam)
OPTIM_REGISTERY.register(torch.optim.AdamW)
OPTIM_REGISTERY.register(torch.optim.Adamax)
OPTIM_REGISTERY.register(torch.optim.Adadelta)
OPTIM_REGISTERY.register(torch.optim.Adagrad)
OPTIM_REGISTERY.register(torch.optim.SparseAdam)
OPTIM_REGISTERY.register(torch.optim.ASGD)
OPTIM_REGISTERY.register(torch.optim.LBFGS)

## Ranger Optimizer - 
> from https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer/blob/master/ranger/ranger2020.py

In [ ]:
# export
@OPTIM_REGISTERY.register()
class Ranger(Optimizer):
    "from - https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer/blob/master/ranger/ranger.py"
    def __init__(self, params, lr=1e-3, alpha=0.5, k=6, N_sma_threshhold=5,
                 betas=(.95, 0.999), eps=1e-5, weight_decay=0,
                 use_gc=True, gc_conv_only=False):

        # parameter checks
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        if not lr > 0:
            raise ValueError(f'Invalid Learning Rate: {lr}')
        if not eps > 0:
            raise ValueError(f'Invalid eps: {eps}')

        # parameter comments:
        # beta1 (momentum) of .95 seems to work better than .90...
        # N_sma_threshold of 5 seems better in testing than 4.
        # In both cases, worth testing on your dataset (.90 vs .95, 4 vs 5) to make sure which works best for you.

        # prep defaults and init torch.optim base
        defaults = dict(lr=lr, alpha=alpha, k=k, step_counter=0, betas=betas,
                        N_sma_threshhold=N_sma_threshhold, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)

        # adjustable threshold
        self.N_sma_threshhold = N_sma_threshhold

        # look ahead params

        self.alpha = alpha
        self.k = k

        # radam buffer for state
        self.radam_buffer = [[None, None, None] for ind in range(10)]

        # gc on or off
        self.use_gc = use_gc

        # level of gradient centralization
        self.gc_gradient_threshold = 3 if gc_conv_only else 1

    def __setstate__(self, state):
        super(Ranger, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        
        if closure is not None:
            loss = closure()

        # Evaluate averages and grad, update param tensors
        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()

                if grad.is_sparse:
                    raise RuntimeError(
                        'Ranger optimizer does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]  # get state dict for this param

                if len(state) == 0:  # if first time to run...init dictionary with our desired entries
                    # if self.first_run_check==0:
                    # self.first_run_check=1
                    #print("Initializing slow buffer...should not see this at load from saved model!")
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)

                    # look ahead weight storage now in state dict
                    state['slow_buffer'] = torch.empty_like(p.data)
                    state['slow_buffer'].copy_(p.data)

                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(
                        p_data_fp32)

                # begin computations
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                # GC operation for Conv layers and FC layers
                if grad.dim() > self.gc_gradient_threshold:
                    grad.add_(-grad.mean(dim=tuple(range(1, grad.dim())), keepdim=True))

                state['step'] += 1

                # compute variance mov avg
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                # compute mean moving avg
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                buffered = self.radam_buffer[int(state['step'] % 10)]

                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * \
                        state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma
                    if N_sma > self.N_sma_threshhold:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (
                            N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay']
                                     * group['lr'], p_data_fp32)

                # apply lr
                if N_sma > self.N_sma_threshhold:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size *
                                         group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

                # integrated look ahead...
                # we do it at the param level instead of group level
                if state['step'] % group['k'] == 0:
                    # get access to slow param tensor
                    slow_p = state['slow_buffer']
                    # (fast weights - slow weights) * alpha
                    slow_p.add_(self.alpha, p.data - slow_p)
                    # copy interpolated weights to RAdam param tensor
                    p.data.copy_(slow_p)
        return loss

In [ ]:
print(OPTIM_REGISTERY)

Registry of Optimizers:
╒════════════╤══════════════════════════════════════════════╕
│ Names      │ Objects                                      │
╞════════════╪══════════════════════════════════════════════╡
│ SGD        │ <class 'torch.optim.sgd.SGD'>                │
├────────────┼──────────────────────────────────────────────┤
│ RMSprop    │ <class 'torch.optim.rmsprop.RMSprop'>        │
├────────────┼──────────────────────────────────────────────┤
│ Adam       │ <class 'torch.optim.adam.Adam'>              │
├────────────┼──────────────────────────────────────────────┤
│ AdamW      │ <class 'torch.optim.adamw.AdamW'>            │
├────────────┼──────────────────────────────────────────────┤
│ Adamax     │ <class 'torch.optim.adamax.Adamax'>          │
├────────────┼──────────────────────────────────────────────┤
│ Adadelta   │ <class 'torch.optim.adadelta.Adadelta'>      │
├────────────┼──────────────────────────────────────────────┤
│ Adagrad    │ <class 'torch.optim.adagrad.Ada

In [ ]:
# export
def create_optimizer(cfg: DictConfig, params):
    "loaded a optimizer in OPTIM_REGISTERY from `cfg`"
    optimizer = OPTIM_REGISTERY.get(cfg.name)(params=params, **cfg.params)
    _logger.info(f"{cfg.name} loaded from OPTIM_REGISTERY")
    return optimizer

In [ ]:
from omegaconf import OmegaConf

model  = torch.nn.Linear(24, 2, bias=True)
params = model.parameters()

In [ ]:
# cfg = OmegaConf.load("../conf/optimizer/adam.yaml")
cfg = OmegaConf.load("../conf/optimizer/ranger.yaml")
# cfg = OmegaConf.load("../conf/optimizer/adamw.yaml")
# cfg = OmegaConf.load("../conf/optimizer/sgd.yaml")

optimizer = create_optimizer(cfg, params)

[01/31 20:33:46 __main__]: Ranger loaded from OPTIM_REGISTERY


In [ ]:
notebook2script()

Converted 00_core.ipynb.
Converted 01a_data.datasets.ipynb.
Converted 01b_data.datasests_factory.ipynb.
Converted 01c_data.mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03a_optimizers.ipynb.
Converted 03b_schedulers.ipynb.
Converted 04a_models.utils.ipynb.
Converted 04b_models.layers.ipynb.
Converted 04c_models.classifiers.ipynb.
Converted 04d_models.builder.ipynb.
Converted 04e_models.task.ipynb.
Converted 05_callbacks.ipynb.
Converted index.ipynb.
